In [1]:
import sys, time, random, math, cv2, os, copy
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from gym.utils import seeding

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

sys.path.insert(0, '../src')
from robot import Robot

Using TensorFlow backend.
/home/daiane/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/daiane/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/daiane/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/daiane/.local/lib/python3.6/site-packages/tensorflow/python

In [2]:
def cicle(ang):
    return (ang + 2 * math.pi) % (2*math.pi)

def get_distance_from_goal(a,b):
    return np.linalg.norm(a-b)

def get_angular_distance(v1,v2):

    v1_aux = v1 / np.linalg.norm(v1)
    v2_aux = v2 / np.linalg.norm(v2)

    theta = np.arccos(np.clip(np.dot(v1_aux, v2_aux), -1.0, 1.0))
    rot_theta = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])
    theta = theta*180.0/math.pi
    theta = (360.0+theta)%360.0

    if abs(np.dot(np.dot(rot_theta,v2_aux),v1_aux)-1.0) > 1e-2:
        theta = 360.0 - theta

    return theta

In [3]:
class EnvRobot(): 
    def __init__(self, robot, goal):
        self.robot = robot
        self.goal = goal
        self.start_rand = False
        self.random, _ = seeding.np_random(12345)
        self.apply_observation()
        self.actions = [[1,0.2],[0.2,1],[1,1]]
        
    def apply_observation(self):
        current_position = np.array(self.robot.get_current_position()[:-1])
        current_angle = cicle(self.robot.get_current_orientation()[2])

        posRel = np.array(self.goal) - np.array(current_position)
        self.observation = np.array([posRel[0], posRel[1], current_angle])
    
    def apply_action(self, a):
        action = self.actions[a]
        self.robot.set_left_velocity(action[0])
        self.robot.set_right_velocity(action[1])
        
    def step(self, action):
        print(action)
        old_position = np.array(self.robot.get_current_position()[:-1])
        old_angle = cicle(self.robot.get_current_orientation()[2])
        
        self.apply_action(action)
        self.robot.step_simulation()
        obsAnterior = copy.copy(self.observation)
        self.apply_observation()
        
        new_position = np.array(self.robot.get_current_position()[:-1])
        new_angle = cicle(self.robot.get_current_orientation()[2])
        
        distance_old = get_distance_from_goal(old_position, self.goal)
        distance_new = get_distance_from_goal(new_position, self.goal)
        
        angle_erro_old = get_angular_distance(self.goal - old_position, np.array([math.cos(old_angle),math.sin(old_angle)])) # graus
        angle_erro_new = get_angular_distance(self.goal - new_position, np.array([math.cos(new_angle),math.sin(new_angle)])) # graus
    
        us_distances = robot.read_ultrassonic_sensors()
        flag = False
        for i in us_distances[:8]:
            flag = flag or (i < 0.5)
        
        reward = (distance_old - distance_new) + ( (angle_erro_old/180) - (angle_erro_new/180) )
        done = distance_new < 0.2
            
        return self.observation, reward, done, {}
        
    def reset(self):
        if self.robot.get_connection_status() != -1:
            positions = [[-2.5,-0.175,0.13868],[2.475,0.5,0.13868],[-0.15,-3.875,0.13868]] # positions begin
            pos = np.random.randint(len(positions))
            tupla = [ [], positions[pos], [], bytearray() ]
            self.robot.call_childscript_function(tupla)
        else:
            self.robot.start_simulation()
            
        if self.start_rand:
            action = self.random.uniform(low=0.2, high=0.5, size=(2,))
            self.apply_action(action)
            self.robot.step_simulation()
        
        self.apply_observation()
        return self.observation
        

In [4]:

path = 'dqn_weights.h5f'
robot = Robot("#0")
goal = np.array([13,0])

env = EnvRobot(robot, goal)

n_actions = 3

model = Sequential()
model.add(Flatten(input_shape=(1,) + (n_actions,)))
model.add(Dense(8))
model.add(Activation('relu'))
model.add(Dense(8))
model.add(Activation('relu'))
model.add(Dense(8))
model.add(Activation('relu'))
model.add(Dense(n_actions))
model.add(Activation('linear'))
print(model.summary())

memory = SequentialMemory(limit=5000000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=n_actions, memory=memory, nb_steps_warmup=32,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

if os.path.isfile(path):
    print("Carregando modelo...")
    dqn.load_weights(path)

dqn.fit(env, nb_steps=2000, nb_max_episode_steps=200, visualize=False, verbose=2)

dqn.save_weights(path, overwrite=True)

#dqn.test(env, nb_episodes=1,nb_max_episode_steps=200, visualize=False)


Connected to remoteApi server.
 Pioneer_p3dx_ultrasonicSensor1#0 connected.
 Pioneer_p3dx_ultrasonicSensor2#0 connected.
 Pioneer_p3dx_ultrasonicSensor3#0 connected.
 Pioneer_p3dx_ultrasonicSensor4#0 connected.
 Pioneer_p3dx_ultrasonicSensor5#0 connected.
 Pioneer_p3dx_ultrasonicSensor6#0 connected.
 Pioneer_p3dx_ultrasonicSensor7#0 connected.
 Pioneer_p3dx_ultrasonicSensor8#0 connected.
 Pioneer_p3dx_ultrasonicSensor9#0 connected.
 Pioneer_p3dx_ultrasonicSensor10#0 connected.
 Pioneer_p3dx_ultrasonicSensor11#0 connected.
 Pioneer_p3dx_ultrasonicSensor12#0 connected.
 Pioneer_p3dx_ultrasonicSensor13#0 connected.
 Pioneer_p3dx_ultrasonicSensor14#0 connected.
 Pioneer_p3dx_ultrasonicSensor15#0 connected.
 Pioneer_p3dx_ultrasonicSensor16#0 connected.
 Vision sensor connected.
 Laser connected.
 Left motor connected.
 Right motor connected.
 Robot connected.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape    